- csv と transformers ライブラリがインポートされ、transformers ライブラリから BertTokenizer と BertModel クラスを使用して BERT トークナイザとモデルがロードされています。
- 2212.csv ファイルが読み取りモードで開かれ、ファイルを読み取るために CSV リーダー オブジェクトが作成されます。
- ヘッダ行は next 関数を用いてスキップされます。
- CSV ファイルの行を繰り返し処理するために、for ループが使用されます。各行について、日付、文、ユーザ名の値が抽出される。
- 文は、BERT トークナイザーを使用して BERT 入力として符号化され、文の BERT 埋め込みは、BERT モデルを使用して計算されます。
-  文の BERT 埋め込みは、モデル出力から抽出される。
- 文の埋め込みと参照埋め込みとの間のコサイン類似度は、PyTorchのnnモジュールのCosineSimilarityクラスを使って計算される。
- 日付、文、ユーザ名、類似度の値が出力される。

In [1]:
import pandas as pd
import torch
from transformers import BertJapaneseTokenizer, BertModel
import re

MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
model = BertModel.from_pretrained(MODEL_NAME)


Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
path = "/Users/iomacbookair2/Documents/lab/DEIM2023/tweet_csv/221213_ann_tue.csv"
df = pd.read_csv((path))
df.sort_values(by = 'created_at', ascending = True, inplace = True)
df = df.reset_index(drop=True)
df['created_at'] = pd.to_datetime(df['created_at'])
df = df.drop("author_id", axis=1)
df = df.drop("username", axis=1)

df
# df.to_csv("sorted.csv", index=False)

,created_at,text
0,2022-12-14 01:00:00,#星野源ANN
1,2022-12-14 01:00:00,#星野源ANN
2,2022-12-14 01:00:00,#星野源ANN
3,2022-12-14 01:00:00,#星野源ANN
4,2022-12-14 01:00:00,#星野源ANN
...,...,...
3343,2022-12-14 03:03:54,年末かぁー。 これからいろいろお楽しみあるから、あっという間に1月よね♪♪ 土井さん楽しみだ...
3344,2022-12-14 03:03:59,#星野源ANN 国性のハガキって手元に何枚か来てるんやね。
3345,2022-12-14 03:04:28,お疲れさま( ^^) _U~~ 今夜もありがとうございました♡ #星野源ANN
3346,2022-12-14 03:04:33,残HP0.2！！！！おうち帰って充電してくれ！！！🥺 #星野源ANN


In [3]:
def remove_urls_and_hashtags(text):
    # Remove URLs
    text = re.sub(r'https?://\S+', '', text)
    # Remove words starting with "#"
    text = re.sub(r'#\S+', '', text)
    text = re.sub(r'＃\S+', '', text)
    return text


df['text'] = df['text'].apply(remove_urls_and_hashtags)
df
# df.to_csv('filtered.csv', index=False)


,created_at,text
0,2022-12-14 01:00:00,
1,2022-12-14 01:00:00,
2,2022-12-14 01:00:00,
3,2022-12-14 01:00:00,
4,2022-12-14 01:00:00,
...,...,...
3343,2022-12-14 03:03:54,年末かぁー。 これからいろいろお楽しみあるから、あっという間に1月よね♪♪ 土井さん楽しみだ...
3344,2022-12-14 03:03:59,国性のハガキって手元に何枚か来てるんやね。
3345,2022-12-14 03:04:28,お疲れさま( ^^) _U~~ 今夜もありがとうございました♡
3346,2022-12-14 03:04:33,残HP0.2！！！！おうち帰って充電してくれ！！！🥺


In [4]:
groups = df.groupby(pd.Grouper(key='created_at', freq='min'))
df_texts_by_minute = pd.DataFrame({
    "texts_by_minute": groups.apply(lambda x: x["text"].tolist())
})
df_texts_by_minute = df_texts_by_minute.reset_index()
df_texts_by_minute
# df_texts_by_minute.to_csv("texts_by_minute.csv", index=False)

,created_at,texts_by_minute
0,2022-12-14 01:00:00,"[, , , , , 星野源のオールナイトニッポン | ニッポン放送 | 2022/12/1..."
1,2022-12-14 01:01:00,"[ , リハーサルはじまったのね。, , リハーサル！😆 , お疲れなんですね。 , ..."
2,2022-12-14 01:02:00,"[リハか！お疲れ様です、、 , ｲｹﾎﾞ... , I'm tired おおお、お疲れ様で..."
3,2022-12-14 01:03:00,"[ほんまに源さんお疲れなのね😢 でも、がんばれぇ～📣 落ち着いてね。 , キス事変ｗｗｗ ..."
4,2022-12-14 01:04:00,"[リハざんまいからのANN。お疲れモードの源さん。アンニュイな感じもいいと思います , とあ..."
...,...,...
120,2022-12-14 03:00:00,"[エンディング , アワード行きが多すぎなラジオ最高ww リハもお忙しそうだし、HP0.2に..."
121,2022-12-14 03:01:00,"[お疲れ様でした☺️ HP0.2なのにがんばってくれてありがとうございました！ , お疲れ..."
122,2022-12-14 03:02:00,"[HP0.2だけど最後までやれて良かった！おうち帰ってフル回復してね！ , お疲れさまでし..."
123,2022-12-14 03:03:00,[お疲れ様でした！今日はお疲れ源さんでしたが、こういうトーンの声は最高に好きです。来週は土井...


In [5]:
def sentence_to_vector(model, tokenizer, sentence):
    tokens = tokenizer(sentence, add_special_tokens=True)["input_ids"]
    input = torch.tensor(tokens).unsqueeze(0)
    with torch.no_grad():
        outputs = model(input, output_hidden_states=True)
        last_hidden_state = outputs[0][:, 0, :]
        averaged_hidden_state = last_hidden_state.mean(dim=0).unsqueeze(0)
    return averaged_hidden_state


def calc_similarity(sentence1, sentence2):
    print("{}\n{}".format(sentence1, sentence2))

    sentence_vector1 = sentence_to_vector(model, tokenizer, sentence1)
    sentence_vector2 = sentence_to_vector(model, tokenizer, sentence2)

    # Reshape the tensors to 1D
    sentence_vector1 = sentence_vector1.reshape(-1)
    sentence_vector2 = sentence_vector2.reshape(-1)

    similarity = float(torch.nn.functional.cosine_similarity(
        sentence_vector1, sentence_vector2, dim=0).detach().numpy().copy())
    print("Similarity:", similarity)

    return similarity


def calc_average_similarity(sentences):
    similarities = []
    for i in range(1, len(sentences)):
        similarity = calc_similarity(sentences[0], sentences[i])
        similarities.append(similarity)
    average_similarity = sum(similarities) / len(similarities)
    # print("Similarities:", similarities)
    print("Average similarity:", average_similarity)

    return average_similarity

In [6]:
df_texts_by_minute["average_similarity"] = df_texts_by_minute["texts_by_minute"].apply(
    calc_average_similarity)
df_texts_by_minute




Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209

星野源のオールナイトニッポン | ニッポン放送 | 2022/12/13/火  25:00-27:00     
Similarity: 0.7311848998069763


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209

一週間のたのしみ😌❕ 今日もこんばんは(*´-`)  
Similarity: 0.6533334255218506


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615

,created_at,texts_by_minute,average_similarity
0,2022-12-14 01:00:00,"[, , , , , 星野源のオールナイトニッポン | ニッポン放送 | 2022/12/1...",0.929364
1,2022-12-14 01:01:00,"[ , リハーサルはじまったのね。, , リハーサル！😆 , お疲れなんですね。 , ...",0.816467
2,2022-12-14 01:02:00,"[リハか！お疲れ様です、、 , ｲｹﾎﾞ... , I'm tired おおお、お疲れ様で...",0.708912
3,2022-12-14 01:03:00,"[ほんまに源さんお疲れなのね😢 でも、がんばれぇ～📣 落ち着いてね。 , キス事変ｗｗｗ ...",0.750233
4,2022-12-14 01:04:00,"[リハざんまいからのANN。お疲れモードの源さん。アンニュイな感じもいいと思います , とあ...",0.727094
...,...,...,...
120,2022-12-14 03:00:00,"[エンディング , アワード行きが多すぎなラジオ最高ww リハもお忙しそうだし、HP0.2に...",0.595660
121,2022-12-14 03:01:00,"[お疲れ様でした☺️ HP0.2なのにがんばってくれてありがとうございました！ , お疲れ...",0.834199
122,2022-12-14 03:02:00,"[HP0.2だけど最後までやれて良かった！おうち帰ってフル回復してね！ , お疲れさまでし...",0.799106
123,2022-12-14 03:03:00,[お疲れ様でした！今日はお疲れ源さんでしたが、こういうトーンの声は最高に好きです。来週は土井...,0.840669


In [7]:
df_texts_by_minute.to_csv('cosine_similarity_221213_ann_tue.csv', index=False)